# Forms

Django has support for designing forms containing user input.  Let's make a start by creating a new project:

In [2]:
mkdir workspace3
django-admin startproject myproject workspace3
cd workspace3
tree .

.
├── manage.py
└── myproject
    ├── asgi.py
    ├── __init__.py
    ├── settings.py
    ├── urls.py
    └── wsgi.py

2 directories, 6 files


Now create our webapp:

In [3]:
python manage.py startapp forms
tree -I __pycache__ .

.
├── forms
│   ├── admin.py
│   ├── apps.py
│   ├── __init__.py
│   ├── migrations
│   │   └── __init__.py
│   ├── models.py
│   ├── tests.py
│   └── views.py
├── manage.py
└── myproject
    ├── asgi.py
    ├── __init__.py
    ├── settings.py
    ├── urls.py
    └── wsgi.py

4 directories, 13 files


Register our app

In [4]:
sed -i -e "/INSTALLED_APPS/ a \    'forms'," myproject/settings.py
sed -n '/^INSTALLED_APPS/,/^]/p' myproject/settings.py

INSTALLED_APPS = [
    'forms',
    'django.contrib.admin',
    'django.contrib.auth',
    'django.contrib.contenttypes',
    'django.contrib.sessions',
    'django.contrib.messages',
    'django.contrib.staticfiles',
]


We begin by defining a form:

In [5]:
cat << EOF > forms/forms.py
from django import forms

# creating a form 
class InputForm(forms.Form):
    first_name = forms.CharField(max_length = 200, initial="Chris")
    last_name = forms.CharField(max_length = 200, initial="Seddon")
    city = forms.CharField(max_length = 200, initial="London")
    country = forms.CharField(max_length = 200, initial="UK")
    roll_number = forms.IntegerField(
		help_text = "Enter 6 digit roll number",
        initial=523681
	)
EOF


forms/     manage.py  myproject/ 


Now create our view:

In [6]:
cat << EOF > forms/views.py
from django.http import HttpResponse
from django.shortcuts import render
from .forms import InputForm

# Create your views here.
def homeView(request):
    context = {}
    context['form']= InputForm()
    return render(request, "home.html", context)

def printResults(request):
    context = {
        'first_name':  request.POST.get('first_name'), 
        'last_name':   request.POST.get('last_name'),
        'city':        request.POST.get('city'),
        'country':     request.POST.get('country'),
        'roll_number': request.POST.get('roll_number'),
    }
    return render(request, "mytemplate.html", context)
EOF

Now set up urls:

In [7]:
cat << EOF > forms/urls.py
from django.urls import path

from . import views

urlpatterns = [
    # called from the template: action = /forms/printResults/'
    path('printResults/', views.printResults, name='printResults'),

    # firefox 'http://localhost:7000/forms/home/'
    path('home/', views.homeView, name='homeView'),
]
EOF

Update the project urls.

In [8]:
cat << EOF > myproject/urls.py
from django.contrib import admin
from django.urls import include, path

urlpatterns = [
    path('forms/', include('forms.urls')),
]
EOF

The template for the form.  Note you need to use a Cross-site request forgery token to prove the target of the POST is genuine.  
https://stackoverflow.com/questions/7064745/how-to-include-external-css-image-etc-in-django-template

In [9]:
mkdir -p forms/templates
cat << EOF > forms/templates/home.html
<!DOCTYPE html>
<html>
<head>
<link rel="stylesheet" type= "text/css" href= "{{ url_for('static',filename='styles/mystyles.css') }}">
</head>
<body>
<form action = "/forms/printResults/" method = "post">
    {% csrf_token %}
    {{form }}
	<input type="submit" value=Submit>
</form>
</body>
EOF

Create the stylesheet

In [10]:
mkdir -p static/styles

cat << EOF > static/styles/mystyles.css
body {background-color: powderblue;}
table, th, td {
  border: 1px solid;
}
EOF

The template for results:

In [11]:
cat << EOF > forms/templates/mytemplate.html
<!DOCTYPE html>
<html>
<head>
<link rel="stylesheet" type= "text/css" href= "{{ url_for('static',filename='styles/mystyles.css') }}">
</head>

<h1>Results</h1>

<table>
    <tr><td>First Name</td>  <td>{{ first_name }}</td></tr>
    <tr><td>Last Name</td>   <td>{{ last_name }}</td></tr>
    <tr><td>City</td>        <td>{{ city }}</td></tr>
    <tr><td>Country</td>     <td>{{ country }}</td></tr>
    <tr><td>Roll Number</td> <td>{{ roll_number }}</td></tr>
</table>
EOF

The file structure is now:

In [12]:
tree -I __pycache__ .

.
├── forms
│   ├── admin.py
│   ├── apps.py
│   ├── forms.py
│   ├── __init__.py
│   ├── migrations
│   │   └── __init__.py
│   ├── models.py
│   ├── templates
│   │   ├── home.html
│   │   └── mytemplate.html
│   ├── tests.py
│   ├── urls.py
│   └── views.py
├── manage.py
├── myproject
│   ├── asgi.py
│   ├── __init__.py
│   ├── settings.py
│   ├── urls.py
│   └── wsgi.py
└── static
    └── styles
        └── mystyles.css

7 directories, 18 files


Perform migrations

In [13]:
python manage.py migrate

Operations to perform:
  Apply all migrations: admin, auth, contenttypes, sessions
Running migrations:
  Applying contenttypes.0001_initial... OK
  Applying auth.0001_initial... OK
  Applying admin.0001_initial... OK
  Applying admin.0002_logentry_remove_auto_add... OK
  Applying admin.0003_logentry_add_action_flag_choices... OK
  Applying contenttypes.0002_remove_content_type_name... OK
  Applying auth.0002_alter_permission_name_max_length... OK
  Applying auth.0003_alter_user_email_max_length... OK
  Applying auth.0004_alter_user_username_opts... OK
  Applying auth.0005_alter_user_last_login_null... OK
  Applying auth.0006_require_contenttypes_0002... OK
  Applying auth.0007_alter_validators_add_error_messages... OK
  Applying auth.0008_alter_user_username_max_length... OK
  Applying auth.0009_alter_user_last_name_max_length... OK
  Applying auth.0010_alter_group_name_max_length... OK
  Applying auth.0011_update_proxy_permissions... OK
  Applying auth.0012_alter_user_first_name_max_l

Start the server

In [14]:
fuser -k 7000/tcp
python manage.py runserver 7000 &

[1] 55343


Use 'touch' to get Django to reload the view.  If we don't do this the template engine gets confused and will fail to locate our templates.

In [15]:
touch forms/views.py

Watching for file changes with StatReloader
Performing system checks...

System check identified no issues (0 silenced).
December 04, 2024 - 22:22:42
Django version 5.1.3, using settings 'myproject.settings'
Starting development server at http://127.0.0.1:7000/
Quit the server with CONTROL-C.

touch forms/views.py^J


Fire up `firefox` to see the app:

In [16]:
firefox http://localhost:7000/forms/home

/home/chris/workspace/python-course/src/workspace3/forms/views.py changed, reloading.
Watching for file changes with StatReloader
Performing system checks...

System check identified no issues (0 silenced).
December 04, 2024 - 22:22:45
Django version 5.1.3, using settings 'myproject.settings'
Starting development server at http://127.0.0.1:7000/
Quit the server with CONTROL-C.

firefox http://localhost:7000/forms/home^J


clean up:
* kill the server
* remove workspace

In [1]:
cd ..
fuser -k 7000/tcp
rm -r workspace3

rm: cannot remove 'workspace3': No such file or directory


: 1